In [30]:
import sympy as sp
from sympy import *
import numpy as np
from opt_einsum import contract as _contract
import itertools as itt
from IPython.display import display, Latex
init_printing()

def contract(expr,*tensors,**args):
    if contract.dest is not None:
        expr+='->'+contract.dest
    rtval= _contract(expr,*tensors, backend='object',**args)
    return rtval.item() if rtval.shape==() else Array(rtval)
    
class contract_dest:
    def __init__(self, dest):
        self.dest= dest
    def __enter__(self):
        contract.dest,self.dest= self.dest,contract.dest
    def __exit__(self, *args):
        contract.dest= self.dest
contract.dest=None

def simplify(expr):
    expr=sp.simplify(expr, ratio=1.7)
    expr=trigsimp(expr, method='fu')
    return expr

def print_nonzero_terms(v):
    for i in itt.product(*[range(d) for d in v.shape]):
        if v[i]!=0:
            print(i,v[i])
def get_covariant_derivative(coords, Gm=None, w=None):
    def covariant_derivative(v,indice_positions=''):
        ''' a: tensor, indice_positions: list of index type
            index types:
                - 'i': internal, 
                - 'u': curved upper, 'd' curved lower,
                - 'U': flat upper, 'D' flat lower
        '''
        rtval=derive_by_array(v,coords)
        for i,ind in enumerate(indice_positions):
            if ind=='u': # D_u T^v=∂_u T^v+Γ^v_ur T^r
                rtval+=np.swapaxes(contract('vur,r...->uv...',Gm,np.swapaxes(v,0,i)),1,i+1)
            elif ind=='d': # D_u T_v=∂_u T_v-Γ^r_uv T^r
                rtval-=np.swapaxes(contract('ruv,r...->uv...',Gm,np.swapaxes(v,0,i)),1,i+1)
            elif ind=='U': # D_u T^m=∂_u T^m+w_u^m_n T^n
                rtval+=np.swapaxes(contract('umn,n...->um...',w,np.swapaxes(v,0,i)),1,i+1)
            elif ind=='D': # D_u T_m=∂_u T_m-w_u^n_m T_n
                rtval-=np.swapaxes(contract('unm,n...->um...',w,np.swapaxes(v,0,i)),1,i+1)
        return Array(simplify(rtval))
    return covariant_derivative

In [31]:
t,r,th,phi=coords=symbols('t,r,theta,phi')
f=Function('f')(r)
df=diff(f,r)
ddf=diff(df,r)

# (co)vielbein e_u^m
signature=Array([-1,1,1,1])
e=diag(f,1/f,r,r*sin(th))
einv=sp.simplify(e.inv())
de=derive_by_array(e,coords)


# metric tensor g_uv=e^u_m e^v_n eta_mn
g=Matrix(contract('um,vm,m',e,e,signature))
ginv=sp.simplify(g.inv())
dg=derive_by_array(g,coords)

# christoffel symbols Γ^k_ij=1/2 g^kl (∂_i g_jl+∂_j g_il-∂_l g_ij)
with contract_dest('kij'):
    Gm=simplify((contract('kl,ijl',ginv,dg)+contract('kl,jil',ginv,dg)-contract('kl,lij',ginv,dg))/2)
dGm=derive_by_array(Gm,coords)
# one might add cotorsion tensor

# spin connection w_u^m_n=-e_n^v ∂_u e_v^m + Gm^r_uv e_r^m e_n^v
with contract_dest('umn'):
    w=simplify(-contract('nv,uvm',einv,de)+contract('ruv,rm,nv',Gm,e,einv))
dw=derive_by_array(w,coords)

D=get_covariant_derivative(coords, Gm=Gm, w=w)
# display(D(e,'dU'))
# display(D(g,'dd'))

# Riemann Tensor R^r_suv=∂u Γ^r_vs - ∂v Γ^r_us +Γ^r_ul Γ^l_vs - Γ^r_vl Γ^l_us
with contract_dest('rsuv'):
    Rie=simplify(contract('urvs',dGm)-contract('vrus',dGm)+contract('rul,lvs',Gm,Gm)-contract('rvl,lus',Gm,Gm))


# Riemann Tensor R^m_nuv=∂u w_v^m_n - ∂v w_u^m_n +w_u^m_k w_v^k_n - w_v^m_k w_u^k_n
with contract_dest('mnuv'):
    RieF=simplify(contract('uvmn',dw)-contract('vumn',dw)+contract('umk,vkn',w,w)-contract('vmk,ukn',w,w))
# RieF2=contract('rsuv,rm,ns->mnuv',Rie,e,einv)

# Ricci Tensor R_uv=R^r_urv
Ric=simplify(contract('rurv->uv',Rie))

# Ricci Tensor R^uv=R^m_nrv e_m^r e_u^n
RicF=simplify(contract('mnrv,mr,un->uv',RieF,einv,e))
